In [120]:
%pip install selenium sentence-transformers --quiet

DEPRECATION: Loading egg at /opt/anaconda3/envs/embed/lib/python3.12/site-packages/sacremoses-0.0.43-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [121]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from collections import defaultdict
from pdfminer.high_level import extract_text
import io
import pickle
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [18]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from collections import defaultdict

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url):
    driver.get(url)
    
    pdf_links_by_year = defaultdict(list)
    
    while True:
        # Parse the page content
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract PDF links and their respective years
        for item in soup.find_all('li', class_='ais-Hits-item nr-tl__result-item'):
            date_div = item.find('div', class_='nr-tl__item-date')
            if date_div:
                date_text = date_div.get_text()
                print(f"Found date: {date_text}")  # Debugging line to check the date text
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year
            
            # Print the entire item structure to debug why PDF links are not being found
            #print(f"Item HTML: {item.prettify()}")  # Debugging line to check item structure

            # Extracting all 'a' tags within the item
            for a_tag in item.find_all('a', href=True):
                href = a_tag['href']
                full_url = f"https://www.bp.com{href}" if href.startswith('/') else href
                print(f"Found PDF link: {full_url}")  # Debugging line to check found links
                if full_url not in pdf_links_by_year[year]:
                    pdf_links_by_year[year].append(full_url)
                    print(f"Added url link: {full_url} under year {year}")  # Debugging line to check added links
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.nr-tl__pagination-next a'))
            )
            next_button.click()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return pdf_links_by_year

# Example usage
bp_news_url = 'https://www.bp.com/en/global/corporate/news-and-insights/press-releases.html'
pdf_links_by_year = read_urls(bp_news_url)

# Print the extracted PDF links by year
for year, links in pdf_links_by_year.items():
    print(f"Year: {year}")
    for link in links:
        print(f"  Extracted PDF link: {link}")

# Close the WebDriver
driver.quit()

Found date: 15 July 2024
Extracted year: 2024
Found PDF link: https://www.bp.com/en/global/corporate/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.html
Added url link: https://www.bp.com/en/global/corporate/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.html under year 2024
Found PDF link: https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.pdf
Added url link: https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.pdf under year 2024
Found date: 15 July 2024
Extracted year: 2024
Found PDF link: https://www.bp.com/en/global/corporate/news-and-insights/press-releases/ipcei-funding-approval-paves-way-for-progress-at-bp-lingen-green-hydrogen-project.html
Added url link: h

In [19]:
pdf_links_by_year

defaultdict(list,
            {'2024': ['https://www.bp.com/en/global/corporate/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.html',
              'https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.pdf',
              'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/ipcei-funding-approval-paves-way-for-progress-at-bp-lingen-green-hydrogen-project.html',
              'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-to-join-adnocs-ruwais-lng-development.html',
              'https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/news-and-insights/press-releases/bp-to-join-adnocs-ruwais-lng-development.pdf',
              'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/second-quarter-2024-trading-statement.html',
              'htt

In [61]:
pdf_links_by_year["2010"]

['https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-agrees-to-sell-upstream-interests-in-pakistan-to-united-energy-group-limited.html',
 'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-agrees-to-sell-its-interests-in-pan-american-energy-to-bridas-corporation.html',
 'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-awarded-exploration-block-in-indonesia.html',
 'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-makes-a-significant-deep-gas-discovery-in-egypts-west-nile-delta.html',
 'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-suspends-production-from-rhum-gas-field.html',
 'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-agrees-sale-of-five-southern-african-marketing-businesses-to-puma-energy.html',
 'https://www.bp.com/en/global/corporate/news-and-insights/press-releases/brendan-nelson-and-frank-l-skip-bowman-to-join-the-bp-

In [95]:
def extract_text_from_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # List of potential content div classes
        content_div_classes = [
            'nr-text-component nr-component',
            'RTE',
            'nr-text-component',
            'content__article-body from-content-api js-article__body'
        ]

        content = ''
        for class_name in content_div_classes:
            content_divs = soup.find_all('div', class_=class_name)
            if content_divs:
                for div in content_divs:
                    paragraphs = div.find_all('p')
                    if paragraphs:
                        content += ' '.join(p.get_text(separator=' ', strip=True) for p in paragraphs)

        return content if content.strip() else None
    except Exception as e:
        print(f"Error processing HTML URL {url}: {e}")
        return None


def extract_text_from_pdf(url):
    try:
        response = requests.get(url)
        with io.BytesIO(response.content) as pdf_file:
            text = extract_text(pdf_file)
            text = ' '.join(text.split())  # Minimize whitespace
        return text
    except Exception as e:
        print(f"Error extracting text from PDF {url}: {e}")
        return ""

def extract_all_text(links_by_year):
    news_by_year = {}  # Use a normal dictionary
    
    for year, links in links_by_year.items():
        news_by_year[year] = []  # Initialize an empty list for each year
        for link in links:
            if 'bp-annual-report' in link:
                print(f"Skipping URL due to keyword 'bp-annual-report': {link}")
                continue

            print(f"Processing URL: {link}")
            if link.endswith('.pdf'):
                pdf_text = extract_text_from_pdf(link)
                if pdf_text:  # Only append if text was successfully extracted
                    news_by_year[year].append(pdf_text)
            else:
                html_text = extract_text_from_html(link)
                if html_text:  # Only append if text was successfully extracted
                    news_by_year[year].append(html_text)
    
    return news_by_year


In [101]:
news_by_year = extract_all_text(pdf_links_by_year)

Processing URL: https://www.bp.com/en/global/corporate/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.html
Processing URL: https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/news-and-insights/press-releases/audi-and-bp-team-up-with-strategic-partnership-for-formula-1.pdf
Processing URL: https://www.bp.com/en/global/corporate/news-and-insights/press-releases/ipcei-funding-approval-paves-way-for-progress-at-bp-lingen-green-hydrogen-project.html
Processing URL: https://www.bp.com/en/global/corporate/news-and-insights/press-releases/bp-to-join-adnocs-ruwais-lng-development.html
Processing URL: https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/news-and-insights/press-releases/bp-to-join-adnocs-ruwais-lng-development.pdf
Processing URL: https://www.bp.com/en/global/corporate/news-and-insights/press-releases/second-quarter-2024-trading-statement.html
Processing URL: https://www.bp.com/content/dam/bp/

In [102]:
news_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010'])

In [116]:
len(news_by_year["2010"])

23

In [104]:
for year in news_by_year:
    print(f"{year}: Number of articles = {len(news_by_year[year])}")

2024: Number of articles = 48
2023: Number of articles = 96
2022: Number of articles = 119
2021: Number of articles = 179
2020: Number of articles = 115
2019: Number of articles = 116
2018: Number of articles = 108
2017: Number of articles = 70
2016: Number of articles = 65
2015: Number of articles = 51
2014: Number of articles = 77
2013: Number of articles = 117
2012: Number of articles = 102
2011: Number of articles = 95
2010: Number of articles = 23


## Splitting up

In [110]:
def split_up_text(article, n): #husk at n må være mindre enn 512 (helst 500) for at modellen skal bruke hele!
    words = article.strip().split()
    if len(words) <= n :
        return [article]
    else:
        new_article = []
        for i in range(0, len(words), n):
            chunk = words[i:i+n]
            new_article.append(' '.join(chunk))
        return new_article

In [111]:
def split_articles_content_by_year(news_by_year, n):
    split_articles_by_year = {}  # Define inside the function

    for year, articles in news_by_year.items():
        split_articles_by_year[year] = []
        for article in articles:
            split_content = split_up_text(article, n)
            split_articles_by_year[year].extend(split_content)

    return split_articles_by_year  # Return the dictionary

split_content = split_articles_content_by_year(news_by_year,450)

In [119]:
with open("BP_split_news_content_by_year.pkl","wb") as file:
    pickle.dump(split_content,file)

## Embedding

In [122]:
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

def embed_text(t): # obs dette tar lang tid
    embeddings = model.encode(t, convert_to_tensor=True)
    return embeddings

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [124]:
print(split_content.keys())
num_years = len(list(split_content.keys()))
print(f"Dette vil ta {(120*num_years)/60} minutter")

import time

annual_embeddings = {}
all_embeddings = []
for year, articles in split_content.items():
    print(year)
    annual_embeddings[year] = []
    start_time = time.time()  # Record start time
    annual_embeddings[year] = []
    for article in articles:
        if time.time() - start_time > 120:  # Check if more than 5 minutes have passed (300 seconds)
            print(f"Skipping year {year} due to time constraint.")
            break
        embedding = embed_text(article)
        annual_embeddings[year].append(embedding)

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010'])
Dette vil ta 30.0 minutter
2024
Skipping year 2024 due to time constraint.
2023
Skipping year 2023 due to time constraint.
2022
Skipping year 2022 due to time constraint.
2021
Skipping year 2021 due to time constraint.
2020
Skipping year 2020 due to time constraint.
2019
Skipping year 2019 due to time constraint.
2018
Skipping year 2018 due to time constraint.
2017
Skipping year 2017 due to time constraint.
2016
Skipping year 2016 due to time constraint.
2015
Skipping year 2015 due to time constraint.
2014
Skipping year 2014 due to time constraint.
2013
Skipping year 2013 due to time constraint.
2012
Skipping year 2012 due to time constraint.
2011
Skipping year 2011 due to time constraint.
2010


In [136]:
annual_embeddings.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010'])

In [130]:
with open("BP_news_embeddings.pkl","wb") as file:
    pickle.dump(annual_embeddings,file)